# Explorando el INEGI

A lo largo de esta libreta iremos indagando en el siguiente enlace. https://www.inegi.org.mx/programas/
## Descargando Datos
Debido a la dificultad de extraer los datos de forma programática, por el momento los descargaremos manualmente.
## Analizando Datos (SIDS)
Descargando los datos de forma manual, comenzamos con la primer área. El Subsistema de Información Demógrafica y Social. 
### Encuestas
Dentro de las encuestas, encontramos de 3 tipos.
* Regulares.
* Especiales.
* Módulos.
#### Regulares
Estás son las que se realizan de forma periódica.
##### ENADID

Comenzaremos con el Diccionario de Datos del ENADID de 2018, el más reciente.

In [398]:
import pandas as pd
import numpy as np

data = pd.read_excel("fd_enadid18.xlsx", sheet_name= None, skiprows= 5)
sheets = list(data.keys())
sheets

['TVIVIENDA',
 'THOGAR',
 'TSDEM',
 'TMIGRANTE',
 'TFECHISEMB',
 'TMUJER1',
 'TMUJER2']

Lo anterior nos da el nombre de las hojas del archivo excel. Dentro de cada una tenemos variables que debemos recuperar. Para ello primero debemos identificar las pequeñas tablas que contiene una hoja.

In [399]:
home = data[sheets[1]]
copy_home = home.copy()
copy_columns = list(copy_home.columns)
copy_home.rename(columns={copy_columns[0] : 'nna'}, inplace= True)
copy_home['nna'] = copy_home.isna().sum(axis = 1)
copy_home.head(n = 20)

,nna,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,10,(0),Consecutivo de la variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,(1),Clave que identifica de manera única al campo ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,(2),Nombre del campo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,(3),Descripción del contenido del campo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,(4),Tipo de caracteres aceptados en el campo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10,(5),Es el número de caracteres que ocupa el campo.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10,(6),Valores que acepta el campo.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10,(7),Clases que se incluyen en la respuesta de cada...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,(8),Nombre del catalogo que aplica a la variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,(9),Información adicional sobre la variable.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Estas se encuentran en la columna ID. Por lo tanto, debemos encontrar una forma de identificarlos, notemos que hay columnas vacías (con NaN), por lo tanto los nombres que buscamos quitan un NaN dentro de la hoja. 

In [400]:
new_tables = list(copy_home[copy_home['nna'] == (copy_home.shape[1] - 1)].index)
name_tables = []
new_tables

[12, 27, 37, 43, 52, 134]

Una vez localizadas, les daremos forma. Primero recopilamos el nombre de la tabla.

In [401]:
df = copy_home.iloc[new_tables[0]: new_tables[1]]
name_tables.append(list(df['Unnamed: 1'])[0])

Eliminamos las columnas que se mantienen pero no son útiles.

In [402]:
new_cols = list(df.iloc[1])
df.columns = new_cols
df = df.reset_index()
df = df.drop(index=[0,1,2]).drop(columns=['index'])
col = list(df.columns)[1:]
df = df[col].dropna(axis = 0,how='all')
df.set_index('ID')

,Mnemónico,Nombre,Descripción,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,Comentarios
ID,,,,,,,,,,
(0),(1),(2),(3),(4),(5),(6),(7),NaN,(8),(9)
1,UPM,Unidad Primaria de Muestreo,Unidad Primaria de\nMuestreo,Caracter,7,0100001..3260039,NaN,NaN,NaN,NaN
2,VIV_SEL,Vivienda seleccionada,Vivienda seleccionada,Caracter,2,01..10,NaN,NaN,NaN,NaN
3,HOGAR,Consecutivo de hogar,Consecutivo del hogar en la vivienda,Caracter,1,1….5,NaN,NaN,NaN,NaN
4,LLAVE_HOG,Llave de hogar,Clave única de identificación del hogar (UPM+V...,Caracter,10,010000305….326079818,NaN,NaN,NaN,NaN
5,LLAVE_VIV,Llave de vivienda,Clave única de identificación de la vivienda (...,Caracter,9,0100003051….3260798181,NaN,NaN,NaN,NaN


Una vez obtuvimos el producto deseado. Veamos como funciona para el resto de tablas.

In [403]:
df2 = copy_home.iloc[new_tables[1]: new_tables[2]]
name_tables.append(list(df2['Unnamed: 1'])[0])
new_cols = list(df2.iloc[1])
df2.columns = new_cols
df2 = df2.reset_index()
df2 = df2.drop(index=[0,1,2]).drop(columns=['index'])
col = list(df2.columns)[1:]
df2 = df2[col].dropna(axis = 0,how='all')
df2 = df2.ffill(axis = 0)
df2

,ID,Mnemónico,Nombre,Descripción,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,Comentarios
3,6,ENT,Entidad federativa,Clave de Entidad Federativa de levantamiento d...,Caracter,2,01…32,01…32,Entidad Federativa,TC_ENTIDADES,NaN
4,7,TAM_LOC,Tamaño de localidad,Clave de tamaño de localidad,Caracter,1,1…4,1,Localidad de 100 000 y más habitantes,TC_ENTIDADES,NaN
5,7,TAM_LOC,Tamaño de localidad,Clave de tamaño de localidad,Caracter,1,1…4,2,Localidad de 15 000 a 99 999 habitantes,TC_ENTIDADES,NaN
6,7,TAM_LOC,Tamaño de localidad,Clave de tamaño de localidad,Caracter,1,1…4,3,Localidad de 2 500 a 14 999 habitantes,TC_ENTIDADES,NaN
7,7,TAM_LOC,Tamaño de localidad,Clave de tamaño de localidad,Caracter,1,1…4,4,Localidad menor de 2 500 habitantes,TC_ENTIDADES,NaN


In [404]:
df3 = copy_home.iloc[new_tables[2]: new_tables[3]]
name_tables.append(list(df3['Unnamed: 1'])[0])
new_cols = list(df3.iloc[1])
df3.columns = new_cols
df3 = df3.reset_index()
df3 = df3.drop(index=[0,1,2]).drop(columns=['index'])
col = list(df3.columns)[1:]
df3 = df3[col].dropna(axis = 0,how='all')
df3.ffill(axis=0)

,ID,Mnemónico,Nombre,Descripción,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,Comentarios
3,8,P2_5,Total de personas en el hogar,Entonces en este hogar hay … personas?,Caracter,2,01...30,NaN,NaN,NaN,NaN


In [405]:
def gen_df(i):
    df2 = copy_home.iloc[new_tables[i]: new_tables[i + 1]]
    name = list(df2['Unnamed: 1'])[0]
    new_cols = list(df2.iloc[1])
    df2.columns = new_cols
    df2 = df2.reset_index()
    if i == 0:
        df2 = df2.drop(index=[0, 1, 2, 3]).drop(columns=['index'])
    else:
        df2 = df2.drop(index=[0, 1, 2]).drop(columns=['index'])
    col = list(df2.columns)[1:]
    df2 = df2[col].dropna(axis = 0,how='all')
    df2 = df2.ffill(axis = 0)
    df2['Categoría'] = name
    return df2

df_final = pd.DataFrame()

for i in range(len(new_tables) - 1):
    df = gen_df(i)
    df_final = pd.concat([df_final, df])
df_final['origen'] = "Vivienda2"

df_final.to_excel("test.xlsx")